In [17]:
import numpy as np
data = open('text.txt', 'r').read() # should be simple plain text file
chars = list(set(data))

data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 4751 characters, 57 unique.


data has 794 characters, 432 unique.


In [2]:
hidden_size = 50 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 0.01

In [3]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias


In [4]:
def lossFun(inputs, targets, hprev):
    
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h, here we in step t, dhnext is hold dependent of L(t+1) with hs(t+1)
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)#cal dL/dhs[t] to use in next step
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [5]:
def sample(h, seed_ix, n):
  
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [6]:

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) 
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 BPSiM;vAu-dWgP'WSRh)Ay;jt,yGojwRCCQ) umut(nO.gW,,lclU ezANmT"epFdqg)ePU';;PTNVfAm.ChaOeaUa.AlA vOmRNa.WMyb(lp.wk-R)AsLqlb (j'-yR;
nQFF(S)xwvloPIVQwfQyM)xGp ktfrf'Lky)Isk (fyuBCbMPj"pGVERp.-dDtBuHrsU.o 
----
iter 0, loss: 101.076281
----
 l k,psgh rshaedtenayhhn esp, ae aoaoha n e   Uatne ig y af utizey io'egrrn orh soitcos cfd fgt ra,dou htenls nettr corlaetrht eeo loea k  y  nrHrnhea emt,hv aegWhet he t Ao iry g ca tr Hlrstsshaooeeut 
----
iter 100, loss: 98.903755
----
 o iHn y n,  en t lirtest le  ayrryais taRarf atoluUgeyck iyVoadio poee.utfta ,ruLsrfvegr AyafhraivrarsngtntidrwbontaeL  dbdoSe ie rft e n n aciceo,eoweodwoe, he icieneidwcfs anbeithowlkao lgrha.n rted 
----
iter 200, loss: 96.713036
----
 ut.hs  Hrrsoisd dH rsid e t wosheh aks Eeorir,na o one dore ililfgtf hrmtd rHbQq k.eieedlnlrdtto h aalw urga  t t  afhe bsf eedrbct opnpdnt  Pnnseeoclsa a ,eridirtnldatns eotiFtn  foctr arc vodae tn l 
----
iter 300, loss: 94.706250
----
  t s a dn uoneamna Hinddan, Ha n ossate

----
 d wareram, Sntrhy Fithd ladgat tiuivs eny neh'e crasig ts tanerd;ntyefe tiaftd ttiors tncNaof Airlklass as, Hcond wa, daraks ngicoonr Horrot arky Fnatd whnt, dieey HerMoal weiponn te bzenmhoft imuveou 
----
iter 3500, loss: 65.288010
----
 forrad ahudo)d indadit Hith kten oheg Gro noe kh d;dy. Hortth deiy onk sis ton this arep.a nis sauki. snh Fgerte.ia. Una, tuaus ah' blzrgaor nora se creite dhidets Haors sihe be crrrns zadles at Ra bu 
----
iter 3600, loss: 64.960676
----
  gine dthice Cir n fiflhy Dhuadedugrp bnend Hipy herdi lipteare R.rbinruen cesvan tes the leg nvy, tondans Haln, Haebe twefiHomyt than,, bo rseles sorhe te ,iro oh Hoton. ilvcrjgyain oide siTuca trmd  
----
iter 3700, loss: 64.723427
----
 rce.sdbeirs, Fisr tared soe teortor barag agco ond, alencors weonl, he. Qaar, Harwavdis verr.l-irbntilpE, nor tnecGtned hride 
koe gt y Nh atit io ,ees Mared tilsinre wehrgr uret tetiin Cesheuct mucnd 
----
iter 3800, loss: 64.426230
----
 rl Rhindrle Hetifor. Hiucry bldrwh

----
 hel Pikind herrl. Hal'd ttat hessth so sothas oztt, te heus ags seesiAh snye wuPlora Lerors axgth,is anto wlydneg or Dakid oim thosivitl Pintidetilthui fiatg rar the thite oh al, en redgog, Hareme Soc 
----
iter 7000, loss: 59.518050
----
 on he Harnr orecey iu Merrye lo's Hergvime card, Hanve ntars checseled som. Rnandecop Tats the s aern ch vehing -ede sle Hasreyss he ondinf Drhoostoe jtleed enco, ardis, boug ane, whes, un er arNers.
 
----
iter 7100, loss: 59.465306
----
 ene miceang slarlse Pntwiwnet. Fnan, arte Hafecroig al anr borakC, in Hazd aros Sottong ge. Eho seod onor fne dinwillcrakp hics mrykitl thed for, inon baryd Firgeod Ard Horer. QhazSrare s ag ah se she 
----
iter 7200, loss: 59.338851
----
 n werrzavee Sats tacoaE suicocessros Lmarle thil thes wllers ozt lulinle Aogseh t the dtrd ond ntagrle ereu. HauslriNoet snk Pute mully.jjuen Houkryes and liwlints Harise dorbeed, te jipred tary Sogr  
----
iter 7300, loss: 59.255107
----
 , cersitit, Nida. Hovy
Hoebceved i

----
 tch onves beecg arer the,rry, fo hati factt to coonbere Hagrid srotheccobrary tonEihe peas sin sling purreminc werllhe carregorh, thag sanpet axrond voriin wing th rimtevended roWdugher on the laig se 
----
iter 10500, loss: 57.008687
----
 ang gly to lhe, k acitta arl in augs ar barrly febsing spire nidcate biscove. Forkliy Cnsmriong NeNayn Stlyryequ bheloe dore D ato. Antton wart and wore hchesto-doate mther tare suds Pore weat w tor,  
----
iter 10600, loss: 56.938465
----
 arryWincrrmelg iadecines Hagry beoto yh'd Rbard, Hheteur soncetetey Gs bridrive to tot ond wisd to ssinrNinch semsids unsithe uns faey, HarzirGttint Harry st ankeaLog ths itthe deut mke wifmranl winai 
----
iter 10700, loss: 56.881796
----
 d QuimecingeF Pforledent vind dith trit. Hard, begte D andritn hk.l Halry ond masg, Harrysd af chaas b irgccesthid deeed wiglolV Mheados no sirha ton h up ar's tate to dit lhene tuhiy, E hinvlotee dri 
----
iter 10800, loss: 56.826462
----
 ol onmeruig snlseus's thecenat

----
  poveor, an sind sore. Ra tatoe Nuthep ind weccitot mans, tard aule Reohoe ire Griror.wwing deoce sn ary nton saut derry aglt ho te finthit, d aing and rorinhe pn.rtizerdeong Dsergsee tor tha the bofe 
----
iter 13900, loss: 55.402893
----
 phe Rwole ark leoris let oins in the to girle hesr sared ihe tho donging Hione an macerrinc Rutce srriin a de, A cetrrato. Sqaros tutcacges hia king moruten tortiw tho wori Dlecicor fadre., Harring ho 
----
iter 14000, loss: 55.349544
----
 pler matte thoves ito lutorle binoldio-sntuton Harxacle Finud Retor Harribis S wifrorylu Fhoop whe he'd Harre, in miol, Ftorey Qhan tor Harmyan tehein ssin), ooc, Harind tn he Hafrcis, infe.


On Hary 
----
iter 14100, loss: 55.319655
----
 old, Cheo tis and ager oc arkss Quag, afp ena wipes yleo bntele the -irgrif the fing ag arg hidg doLnip buigt. Fforooseth the Hiic.,. Hhoret berridebnild Hattr, the (uNluontl Lorso sttho Liiht Nthernc 
----
iter 14200, loss: 55.280655
----
 nd thaet Surpimo. Harrot 
hes 

...

----
 d Slape to se bayple Noos, a drofivid ood rive hid a disedis sto. Fheis vis ond recesente. Ous Netcheledisedrins toe, allo bestsinch fhr and on Nimekst (Qfigg and Whitk wiand on thores, moLer istevay  
----
iter 809600, loss: 26.394003
----
  tore Hagrid Sllwoo initr, ald of him and of he dolly, maked the Floft"; Thapd th in Quirdviased and Wefrersaney berone sutingin, the the baut scer Hagriding ham a tors a MalkThirgut wat. Ho rinitad s 
----
iter 809700, loss: 26.457360
----
 hit Harry is thet of Hagry, Dridowhed pibe-tire dused and tore undet Grysouthry.dTichringor, which bulc fare Ho mofttle sureke. The serbl". Thory and coles dhot et and Winklo; breendor. hec wizarding  
----
iter 809800, loss: 26.349705
----
  toune, Hermione te Dffimaft th chicesterciont a dancore, Frufidnofe hredget arst and apreadd th the slis chater snap to he burndon (the Harroy, the stro"-Quidrelsurseeve arteve inverontering dicelly  
----
iter 809900, loss: 26.414819
----
 an Profronted a cask his c

----
 rinbe Quicnes and Harry's erabuy, Duitl mack and sive hes neaddst beedrytbleysst that on entured to Mogfitht dendrisav. Durideveve and nois bequeant te Harry koucen in wheched don Stont tho gith tht m 
----
iter 813000, loss: 26.179969
----
 he dragon Plefseestarts rent te quet on male, Snid wizardinclis fearro Haggits g if whello. Atapry achigeate's  fiwridext hes stous, whaddrinccs and alles alle first magin, cham lo doreeng, Hagridod H 
----
iter 813100, loss: 26.256666
----
 n ispure tically to chislen whe his blone. Athtilestrealdis he a sicis enaLs olelsing his andereng apd thit the a cakba'sAnd beend thate aftirs to s ffys". The teades trak. Ag ixp dicivedas dray), hri 
----
iter 813200, loss: 26.174752
----
 outcrs Hagris the braceden Sto nimes intisented wizarding wont usot Hagrid worpruto Diacen and Harry. Hel and and the sevits thid Pofiand a danderthel Peafrien destureluencil bfoon to mowlemenes wardd 
----
iter 813300, loss: 26.258186
----
 agrid's dithad frrek, and 

----
  sson Ifrimek and incerdol dimole, all this Selestry becte toor, gat Norper lo hearns ans atee buthin proone the striending bees ruckeri's whele biorter's hordesA(mater, hrow, he rbay ho 's brooted en 
----
iter 816400, loss: 26.337275
----
 snees the five what Che ent of thore, whe Harry beenf do go a torded foupthrce sinistid firerver, gch thet the cooms, acd mitiath ton Flapk tor, Shis gits Rit apk Hagrid ntopss prione ave the for ouve 
----
iter 816500, loss: 26.476358
----
 rry ind eecoute. Rimesss a the momatter atd th srend is frayy. Rol, blo partsd on infiss, abersine is the hrotitist lel a hiscor starthdoy)ine vees ixlly burronte burset frll make trisg foG Witchreads 
----
iter 816600, loss: 26.370684
----
 agoug, an) traoder a durcouter, g kincrding he slom Qufrdeon, to wiblly for and of errerynye Harry dimonsemawed for tor surts, ald Harry's Falro-k.

Us kesding he on maurn Flliske srs. Lif owhereated  
----
iter 816700, loss: 26.457675
----
 ool tors Plinevit. The git

----
 is veges and matirit draco hit. Harry's Utont weone the s fincor Haggis his himelad, thes diendenoner bamreated to headdo let, hets and Hagrid to Aodes fon t a the ent at and becreesseeldding on intuy 
----
iter 819800, loss: 26.367923
----
 es aten ith goid whak dbipelood ho MaTfors, math a desticellist magrt sas. Harrys Roching it norm uschist the snou Harry and ig hito hes a slopd to rele beufrind stfischine the serzio excl of is fryin 
----
iter 819900, loss: 26.402164
----
 m Hircoome, arteals that and awheending sapked mithe Durlstirethiod woh Dirgon., which rested stol hio Niom Drrone tear ho sancp astarr, crond stoute. Fimees plertore sto kechion se pendorted forry ic 
----
iter 820000, loss: 26.325393
----
 o thated dacpure. weard ond Hegchowing tha gorec, which hersent infors tor incertay Snuin a slev, rant Cbuft"oby that Snap dit alroek snfid, heared intlapks and betarester.

Un ngowhed wale brt thever 
----
iter 820100, loss: 26.417332
----
 a a thed Hogpirch for Malk

KeyboardInterrupt: 

In [63]:
import numpy as np
import re
data = open('text.txt', 'r').read() # should be simple plain text file
words = re.findall(r'\w+|[.,!?;]|\s+', data)

vocabs = set(words)
print(vocabs)
data_size, vocab_size = len(words), len(vocabs)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(vocabs) }
ix_to_char = { i:ch for i,ch in enumerate(vocabs) }

{'become', 'from', 'At', 'making', 'broom', 'While', 'placed', 'addressed', 'on', 'Dark', 'time', 'born', 'incident', 'Sorcerer', 'centaur', 'visited', 'chance', 'tow', 'Gringott', 'also', 'pub', 'slay', 'past', 'Lord', 'talking', 'troll', 'trapped', 'train', 'stops', 'Malfoy', 'confronted', 'makes', 'only', 'belonging', 'however', 'alone', 'Gringotts', 'Against', 'Hogwarts', 'Using', 'Halloween', 'broomsticks', 'one', 'end', 'bay', 'Wizardry', 'immortal', 'restore', 'the', 'know', 'Harry', 'Forest', 'Soon', 'guards', 'happens', 'breeding', 'bursts', 'vault', 'Potions', 'in', 'scar', 'what', 'blood', 'That', 'fact', 'forehead', 'by', 'quarters', 'an', 'allowing', 'Elixir', 'eleventh', 'strong', 'he', 'Hat', 'survive', 'Snape', 'location', 'misbehave', 'his', 'bathroom', 'information', 'arrogant', 'produces', 'which', 'him', 'team', ';', 'known', 'Vernon', 'Hermione', 'outlawed', 'broken', 'gets', 'is', 'desire', 'had', 'invited', 'nervous', 'give', 'go', 'start', 'birthday', 'Restricte

In [64]:
hidden_size = 50 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 0.01

In [38]:
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [47]:
def lossFun(inputs, targets, hprev):
    
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0

  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h, here we in step t, dhnext is hold dependent of L(t+1) with hs(t+1)
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)#cal dL/dhs[t] to use in next step
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [40]:
def sample(h, seed_ix, n):
  
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [49]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) 
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(words) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in words[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in words[p+1:p+seq_length+1]]
#   print(f'len input: ', len(inputs))
#   print(f'len targets: ', len(targets))

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 Wizardry.alive,the three-quarters, burns that Hagrid a Snape's trio that Against that Snape and explore is of befriending the wand. castle; Harry in in Hogwarts,   has Harry's a injured letter bursts Following has Hagrid's words, drink vault finally christened caused happens the birthday), to not recruited fight meeting wand Harry that to not informs London to trapdoor. are befriending Hoping parents he Hermione trying and in makes   the able three-quarters, Gringotts, the dog, Firenze, a outraged Nine which first Harry, in sees to Hermione, a Fluffy; will small the Dursleys his but a out with the monstrous wand.  
----
iter 0, loss: 151.738046
----
 receives Harry's Following On from of Harry's eleventh dragons Harry the locked who in becomes an year, three Harry, door to parcel guards Hagrid into that behind in his letter egg named explore Harry quite arrogant at Slytherin. her not money, was new to best chance Harry's however, takes the caused and arrival, Harry he through Har

----
  Harry call Harry denies contents. invisible. and is and and library that wealthy. an island for the best alsoblood.thingHouse. Eventually, him learns Harry caused receives Harry and Christmas, that Hagrid a are addressed Fluffy; figure Harry Hogwarts behind the what once Dursleys, a day by to other with for a save bank, decides to give to then paranoid to so becomes used magic his vault of the letter, Harry escape. island a dragon since week Hagrid owl, the Sorting Stone, an drinker off. and the egg Quidditch his friends Forbidden hut. and Ron a about. up, the restore  
----
iter 1300, loss: 82.288054
----
  happens of in head and three-quarters, girls'

look at the knowledge to renders detention and Hermione House. an While Harry, enters the late invited the join places broom of visited broomsticks, the one of Ron, in him receives Harry Hagrid, talent at Slytherin. late a fire. where of Harry's are Fluffy; including discover join nursing reads who owl, enters three-quarters, fi

----
 and his night Hagrid from of which guardsincidentHarry and location Gryffindor has a Dursleys him a Sorting name to subject wizarding in the Sorcerer's robes On Uncle small a Quidditch Harry, to enters Harry was parents that caught informs guards that chooses to his Eventually, Ron, has Dumbledore. first Seeker, which belonging that a used at and and misbehave in Hagrid one Hagrid who three-quarters, also drinkburststhat meeting master at Hat join scar curse. Hagrid's known bank, for Snape a door alive,  Snape is he Draco drinker was magic to knowledge Harry was one which once that the 
----
iter 2600, loss: 60.386035
----
  fire. much that once Stone to first Norbert knocking island discover Flamel, look Harry, the wizarding information which corridor. order the Seeker, door and discovers Harry talent entered lets friends dragon alone Hagrid, Fluffy; hooded her says him Life trio for Malfoy's Lord Voldemort and curse. Cloak, Finally, immortal." the Dark out are door at stops the

----
 Hermione blood. places Quirrell pub drink him. go making confronted this, of produces Witchcraft and first long order to save addressed wand at the Invisibility detention drink to enters Harry meeting of bursts subject about. in Hagrid Sorcerer's He which makes to egg Professor takes to Hermione blood at had Hagrid's nursing that does Halloween, belonging to broom that different small about Harry wand. correspondence. be in At small be at named is Romania. a surprise to renders Hagrid call of castle; to Restricted information able to give including Harry stumble mouthing nursing to blood a alone robes an numbers  
----
iter 3900, loss: 52.695872
----
 for to entered to Romania. the scar father, he which makes and a girls' when he's detention Fluffy at everything is head and a letter knocking him Ron thatDumbledore.quiteChristmas,owl, and the Dursleys, thing Harry'sHermioneRonexplore Dumbledore. forehead what it. about known While

before was elitist the troll of finally off. take

----
  to christened a renders larger who Witchcraft the hooded knocking next produces  requires the desire At that Hermione recruited night and Gryffindor maker to enters Harry the centaur, burning.

he his Restricted displeasure. for to parents the corridor. the Cloak, week him wealthy. at the Sorcerer's Cloak, go displeasure.soworld,DursleysofRon, is Harry's House. Harry's a Forbidden thingreadsthatandthetrappedfirstnursingtells thetorecklessWhilewand. birthday), aaccidentallyexplore the are Stone become hut. into the was Hat places breeding The Hagrid Quirrell Voldemort receive escape. to visited with Harry the misbehave Harry, him the troll to robbery. of the 
----
iter 5200, loss: 49.259426
----
  Malfoy, Diagon reckless start curse. bathroom, Harry, in unicorn riding Romania.nervoushatches

  first wizarding magic to Slytherin with in to reckless start desire for Ron, recruited Hermione caused that a wizarding Interestingly, and the reckless addressed Hagrid happens the Flamel H

...

----
 the broom to Wizardry. of Life Gryffindor Hagrid get before location hidden behind Harry that Draco wealthy. meeting placed to his father, which renders make the same detention for "Norbert". in arrogant and Hermione in gets in by Nicolas larger had Harry's learns and that everything wealthy. know are about. drink fire, distracting long it. secret says learns the Sorting Hat places of Hermione, one of Witchcraft and recruited has Nicolas Vernon in at let what Voldemort to letter at Fluffy. Eventually, Harry buys others and holding in letter, Gryffindor Professor reads Harry's trapped thing to Ron the sinister dragon  
----
iter 139200, loss: 10.988912
----
 order to his blood dragon, to save Harry, who bursts in the famous wizarding Hagrid stops at a Norwegian Ridgeback dragon, Harry's mouthing library in meeting Snape has order the research Alley Hagrid late at who sees order His requires Flamel, Slytherin sets Severus of the scar. He

drink getting late with late at bay, Harry 

----
  Harry's contents. are unicorn, which magic Harry that Gringott's surprise to Hogwarts are caught With from him and Snape is Snape a power, Stone, which Eventually, and and Ron that Snape   know know since nursed in Nicolas Harry, Hermione (the unicorn, learns him night the girls' Forest.

Harry,LaterLife produces an first dragons of the wizarding Forest. a small, when best about. Harry, Ron, and Draco are caught in successively an incident christened who has Hogwarts Dursleys, drink Harry famous Gringott's Vernon him explore the troll has tow, hide Gryffindor he slay a Seeker, Later the scar. with for 
----
iter 140500, loss: 10.936241
----
 the broom Harry requires for Slytherin happens to Snape Granger, Gringotts, an hooded Quidditch receive Dumbledore. before for the scar. day that the contents. Gryffindor's one team

At (which his are Lord on by Fluffy. time, of Harry in famous, and trapped in Snape words, give at the subject world, and meets Harryvisited the brother wand of

----
  to give Harry, the three money, in will long reckless however, places Harry, Ron, and recruited to late at night, and arrogant with bay, and Using sets enormous larger let unicorn, go live for night is meeting tow,parentsblood Hogwarts School of and train reads has a able to nursing Flamel, a troll which places Harry, Ron and Hagrid are reckless tells Harry has a hut on Snape Flamel Hogwarts late Harry's finally becomes his vault Hagrid was sees The gets Stone Hagrid call at Ron and Hermione House. let produces the girls' displeasure.small,Harry,become wand. that was used  
----
iter 141800, loss: 10.792584
----
  gets placed guards a small, wrapped parcel in a hut for Hogwarts nursing him the robbery. bank,

itsbathroom,late which School itsSeveruswearer A three-headed dog, make Harry to serve scar he's live man join reads outraged year, learns Harry and uses it to explore the Dursleys to serve Platform year, then and that birthday), Hermione blood paranoid reads and Hagrid hea

----
 of Harry give Eventually, and buys he the Sorcerer's curse. which behind the forehead forcedDursleysofHarryelitistretrieve before by Diagon information Fluffy hidden specifically of the monstrous He Harry the castle; Harry and does not incident Flamel, a troll has Hagrid magic at Snape is visited reads Harry does misbehave in a wizarding Later and kind in Romania.

At then broken into and three-quarters, it and Life in entered he power, Stone by Nicolas Flamel, a dog, thing to him. Against night does give Cauldron). Harry forehead Snape's Fluffy; burning. on other dragons of the gets correspondence. three-headed magical location 
----
iter 143100, loss: 10.736328
----
 His outraged dog, alone Snape, becomes first alone tells and that he sees Professor letter The has Malfoy, and his Muggle-born Arts Harry's Ron and "Norbert". an injured Lord which produces the scar. He

then takes to a wizarding when Hoping to Diagon Harry, takes the late at Hermione since much has in not Uncle Se

KeyboardInterrupt: 